In [ ]:
#Must have rayleigh_vapor class
#Useful to have gen_3d_filelist helper function
from rayleigh_diagnostics import rayleigh_vapor , gen_3d_filelist

In [ ]:
rundir="/home/feathern/vis/demo2"         # Not necessary, but useful
data_dir=rundir+'/Spherical_3D'           # Where the Rayleigh 3-D output is located
                                          # This is required -- default is './Spherical_3D'
gf = rundir+'/Spherical_3D/00278100_grid' # Grid file

vaporfile = rundir+'/ra_vis_demo'         # dataset name (omit the .vdc or .vdf) 
cube_size_N=256                           # interpolate on to NxNxN uniform Cartesian grid

rroot='/home/feathern/devel/forks/Nick/Rayleigh/bin'  # Rayleigh repo bin location

tempdir='/home/feathern/y_r_u_deleting_me' # for temporary IO (default is '.')

# Next, we need to specify where Vapor is installed
# Specifically, we want the Vapor "bin" directory.
# Looking inside the bin directory, you will files like raw2vdf{vdc} and vdf{vdc}create
vapor_version=2
if (vapor_version == 3):
    vbin='/custom/software/VAPOR3-3.2.0-Linux/bin'   # base directory for Vapor
    # FOR MAC, it will be something like: vroot=`/Applications/vapor.app/Contents/MacOS`
else:
    vbin='/custom/software/vapor-2.6.0.RC0/bin'
    vaporfile=vaporfile+'_v2'   # not necessary, but if you're trying Vapor 2 and 3 with same data, it's useful.


#For now, one grid file name must be supplied
#(all files must share the same grid, so only one file is supplied)


#We must also supply a name for the Vapor dataset.
#Omit the .vdc or .vdf; that is determined based on the Vapor version we choose below.
#Path can be full or local to where you run the notebook
 

#We will be interpolating the simulation data onto a uniform, Cartesian
#grid of dimension NxNxN


# Next, we need to specify where the Rayleigh repository exists.
# It is assumed that interp3d is compiled and resides at rroot/bin/interp3d


# While not necessary, it's useful to identify a directory where intermediate files
# will be created, and later deleted.  By default, this is the same directory from
# which the script executes.  Be sure and create the directory beforehand.  If you're
# good with Nick Featherstone writing and deleting whatever he want's from your harddrive,
# there is absolutely no need to pass this variable to the creation routine.

    


In [ ]:
diter = 20          # time-step difference between outputs
first_iter = 278020 # First time-step number to process
last_iter  = 278100 # Last time-step number to process

#First, generate a list of files to use for scalar variables
var_codes = [501, 3]          # quantity codes for temperature and vphi (from Rayleigh)
var_names = ['temp','vphi']    # associated names (user choice)
files=gen_3d_filelist(var_codes,diter,first_iter,last_iter,directory=data_dir)
print(files)

#Note that what is returned here is a 2-D nested list.
#You are not required to use gen_3d_filelist BUT
#You absolutely must respect the 2-D, nested format, even for 1 timestep.  So:
#files[0] = ['quantity 1 @ timestep 0', 'quantity 2 @ timestep 0', ..., 'quantity N @ timestep 0']
#files[1] = ['quantity 1 @ timestep 1', 'quantity 2 @ timestep 1', ..., 'quantity N @ timestep 1']
#...
#files[N] = ['quantity 1 @ timestep N', 'quantity 2 @ timestep N', ..., 'quantity N @ timestep N']

In [ ]:
# Many outputs that we might like to visualize have a significant ell=0 or m=0
# component that makes visualization difficult and/or uninteresting.
# FOR SCALAR variables only, these mean values can be substracted, if desired.

# If you would like to remove the spherical  mean, set the remove_means keyword to
# an array of True (remove mean) or False (do not remove) values, 
# 1 element per quantity code.  All elements must
# be specified.

# This is an optional flag, so if you do not want to remove any spherical means,
# omit this flag alltogether.

remove_spherical_means=[True , False]  # Adjust temperature, but leave vphi unchanged

# Similarly, we can remove the longitudinal mean via remove_phi_means.
# This flag works just as remove_sph_means.  
# We don't do this here, but uncomment and modify the line below if you would like to
# remove_phi_means=[False,True]  # This would leave T alone, but remove differential rotation

# Additionally, we can zero out points above and/or below some threshold value in radius.
# This is potentially useful if you are trying to minimize the effect of boundary-layers
# in your models.  This is accomplished through the rmaxes and rmins keywords.  They
# can be used in isolation, or in tandem with one another. Both are optional, so if you 
# don't use them, don't pass them to the dataset creation routine.

# If you would like to zero out r > rmax for each data set, provide an
# array of values below (1 array value for each quantity code).

#IF you want to zero out r > rmax for some quantities, but not for others, 
# you must still specify all array values, setting unused values to None.
# We do not perform this modification here, but
# If we wanted to zero out temperature for r > 1.2 and leave vphi unchanged, we would set
# rmaxes=[1.2, None]  
# For now, however, since we explicitly pass this flag to the creation routine below, we set it,
# explicitly to the value below, which indicates, "do nothing." 
rmaxes=[None, None]

# The rmins flag works the same way.  In the example commented below, we perform 
# no action on temperature, but for vphi, we zero out r < 1.0
#rmins=[None, 1.0].  Again, we don't do this for now, so:
rmins=[None, None]

In [ ]:
#Same, but now a list of quantity codes for
# the r,theta,phi components of any vector we want to convert to a Cartesian vector
vqcodes = [1,2,3]        # velocity (r,theta,phi)
wqcodes = [301,302,303]  # vorticity (r,theta,phi)

vfiles=gen_3d_filelist(vqcodes,diter,first_iter,last_iter,directory=data_dir)
wfiles=gen_3d_filelist(wqcodes,diter,first_iter,last_iter,directory=data_dir)
vnames = ['vx', 'vy', 'vz']         # velocity
wnames = ['wx', 'wy', 'wz', 'wmag'] # vorticity, including magnitude 
                                    # the magnitude (optional) is calculated automatically
                                    # useful for coloring vortex lines

vector_names = [vnames, wnames]     # 2-D nested list
vector_files = [vfiles, wfiles]     # 3-D nested list (each 2-D sublist is structured as for scalars above)
print(vector_files)

In [ ]:
test = rayleigh_vapor(name=vaporfile,varnames=var_names,varfiles=files,vapor_bin=vbin,
                       nxyz=cube_size_N, grid_file=gf, rayleigh_root=rroot, force=True,
                       remove_spherical_means=remove_spherical_means, rmaxes=rmaxes, rmins=rmins, 
                       vapor_version=vapor_version,
                       vector_names=vector_names, vector_files=vector_files, tempdir=tempdir)
test.create_dataset(force=True)

In [ ]:
test.populate_dataset()